In [525]:
# jupyter nbconvert --to python Extract_Playlist_With_Duplicates.ipynb

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import pandas as pd
from datetime import datetime

import os
from subprocess import Popen

from filter_string import filter_string

In [526]:
print("...Making CSV...")

...Making CSV...


In [527]:
#Authentication - without user
with open(r"C:\Users\mackt\Python\Music Library\Spotify_App_Credentials.txt") as f:
    sac_lines = f.readlines()
    cid = sac_lines[0].split(", ")
    cid = cid[1].split("\n")
    cid = cid[0]
    # print(cid)
    secret = sac_lines[1].split(", ")
    secret = secret[1]
    # print(secret)

In [528]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [529]:
# playlist_link = "https://open.spotify.com/playlist/5F5yHyXHt6vp2taA7DrEhJ?si=b4f82e9651444f28"
playlist_link = "https://open.spotify.com/playlist/7LaicnuGlBjUoHZ5Rd4tjm?si=e9f47ebd992b4d08"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

In [530]:
results = sp.playlist_tracks(playlist_URI)
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

In [531]:
for track in tracks:
    # print(track["track"]["name"])
    pass
# print(sp.audio_analysis(track["track"]["id"]), '\n')
print(sp.audio_features(track["track"]["uri"]), '\n')
print(sp.track(track["track"]["uri"]), '\n')
print(track["track"]["artists"][0]["id"], '\n')
print(sp.artist(track["track"]["artists"][0]["id"])["genres"], '\n')


[{'danceability': 0.845, 'energy': 0.631, 'key': 9, 'loudness': -9.567, 'mode': 1, 'speechiness': 0.217, 'acousticness': 0.456, 'instrumentalness': 2.13e-06, 'liveness': 0.137, 'valence': 0.526, 'tempo': 132.084, 'type': 'audio_features', 'id': '1xxJiB2B3rMw8g85PPtcN7', 'uri': 'spotify:track:1xxJiB2B3rMw8g85PPtcN7', 'track_href': 'https://api.spotify.com/v1/tracks/1xxJiB2B3rMw8g85PPtcN7', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1xxJiB2B3rMw8g85PPtcN7', 'duration_ms': 156373, 'time_signature': 4}] 

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3RukKIu27TAGLJ4VT5mNRz'}, 'href': 'https://api.spotify.com/v1/artists/3RukKIu27TAGLJ4VT5mNRz', 'id': '3RukKIu27TAGLJ4VT5mNRz', 'name': 'JV.JARVIS', 'type': 'artist', 'uri': 'spotify:artist:3RukKIu27TAGLJ4VT5mNRz'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR

In [532]:
# Song Library

sl_tr_id = []

Duplicate_Index = []
Duplicate_ID = []

sl_tr_name = []
sl_tr_name_fil = []
# All uris for each song
sl_tr_uri = []

# All Artists for each song
sl_ar_name = []
sl_ar_uri = []

lib = {}
l = 0
track_id = 1

for track in tracks:
    lib["track{0}". format(l)] = {}

    # Track ID
    lib["track{0}". format(l)]["id"] = l

    # Track Name and URI
    track_uri = track["track"]["uri"]
    track_name = track["track"]["name"]
    track_name_fil = filter_string(track_name)

    lib["track{0}". format(l)]["song"] = track_name
    lib["track{0}". format(l)]["song_fil"] = track_name_fil

    sl_tr_name.append(track_name)
    sl_tr_name_fil.append(track_name_fil)
    sl_tr_uri.append(track_uri)

    # Artists Name and uri
    track_artists_name = []
    track_artists_name_fil = []
    track_artists_uri = []

    lib["track{0}". format(l)]["artists_info"] = {}
    for i in range (10):
        lib["track{0}". format(l)]["artists_info"]["ar_name{0}". format(i)] = track["track"]["artists"][i]["name"]

        lib["track{0}". format(l)]["artists_info"]["ar_uri{0}". format(i)] = track["track"]["artists"][i]["uri"]

        track_artists_name.append(track["track"]["artists"][i]["name"])
        track_artists_uri.append(track["track"]["artists"][i]["uri"])

        i = i + 1
        try: track["track"]["artists"][i]["name"]
        except IndexError:
            y = i-1
            sl_ar_name.append(track_artists_name)
            sl_ar_uri.append(track_artists_uri)

            space = ", "
            str_artists = space.join(track_artists_name)
            lib["track{0}". format(l)]["artists"] = str_artists
            break

    # Track Album
    lib["track{0}". format(l)]["album"] = track["track"]["album"]["name"]

    # Track Release Date + Local
    # Local Tracks
    if track['is_local'] == True:
        lib["track{0}". format(l)]["date_spot"] = 'None'
        lib["track{0}". format(l)]["local"] = 'Yes'
    # Spotify Tracks
    else:
        track_release_date = sp.track(track_uri)["album"]["release_date"]
        if sp.track(track_uri)["album"]["release_date_precision"] == 'day':
            track_release_date = str(datetime.strptime(track_release_date, "%Y-%m-%d").date())
        elif sp.track(track_uri)["album"]["release_date_precision"] == 'month':
            track_release_date = datetime.strptime(track_release_date, "%Y-%m").date()
        elif sp.track(track_uri)["album"]["release_date_precision"] == 'year':
            track_release_date = str(datetime.strptime(track_release_date, "%Y").date())
        lib["track{0}". format(l)]["date_spot"] = track_release_date
        lib["track{0}". format(l)]["local"] = 'No'

    # Track Duration
    feature_aud = sp.audio_features(track_uri)[0]

    if track['is_local'] == True:
        lib["track{0}". format(l)]["time"] = track["track"]["duration_ms"]
    else:
        lib["track{0}". format(l)]["time"] = feature_aud["duration_ms"]

    # Track Popularity
    lib["track{0}". format(l)]["popularity"] = track["track"]["popularity"]

    # Track Audio Features
    # Locals
    if track['is_local'] == True:
        lib["track{0}". format(l)]["danceability"] = 'None'
        lib["track{0}". format(l)]["energy"] = 'None'
        lib["track{0}". format(l)]["key"] = 'None'
        lib["track{0}". format(l)]["loudness"] = 'None'
        lib["track{0}". format(l)]["mode"] = 'None'
        lib["track{0}". format(l)]["speechiness"] = 'None'
        lib["track{0}". format(l)]["acousticness"] = 'None'
        lib["track{0}". format(l)]["instrumentalness"] = 'None'
        lib["track{0}". format(l)]["liveness"] = 'None'
        lib["track{0}". format(l)]["valence"] = 'None'
        lib["track{0}". format(l)]["tempo"] = 'None'
    # Spotify
    else:
        lib["track{0}". format(l)]["danceability"] = feature_aud["danceability"]
        lib["track{0}". format(l)]["energy"] = feature_aud["energy"]
        lib["track{0}". format(l)]["key"] = feature_aud["key"]
        lib["track{0}". format(l)]["loudness"] = feature_aud["loudness"]
        lib["track{0}". format(l)]["mode"] = feature_aud["mode"]
        lib["track{0}". format(l)]["speechiness"] = feature_aud["speechiness"]
        lib["track{0}". format(l)]["acousticness"] = feature_aud["acousticness"]
        lib["track{0}". format(l)]["instrumentalness"] = feature_aud["instrumentalness"]
        lib["track{0}". format(l)]["liveness"] = feature_aud["liveness"]
        lib["track{0}". format(l)]["valence"] = feature_aud["valence"]
        lib["track{0}". format(l)]["tempo"] = feature_aud["tempo"]

    lib["track{0}". format(l)]["uri"] = track_uri

    # Duplicate Songs
    lib["track{0}". format(l)]["duplicates"] = {}
    lib["track{0}". format(l)]["duplicates"]["dup_id"] = -1
    lib["track{0}". format(l)]["duplicates"]["num_dup"] = 0

    track_uris = [track_uri]
    current_index = len(lib)

    print(track_name)
    same_tr_names = sl_tr_name_fil.count(track_name_fil)-1
    dup = 0
    if same_tr_names != 0:
        last_ind = 0
        # print(same_tr_names, "same name")
        for i in range(same_tr_names):
            index = sl_tr_name_fil.index(track_name_fil, last_ind)
            if sl_ar_uri[index] == track_artists_uri:
                if dup == 0:
                    dup = 1
                    dup_id = index
                    Duplicate_Index.append(dup_id)
                    Duplicate_Index[index] = dup_id

                    lib["track{0}". format(l)]["duplicates"]["num_dup"] = dup
                    lib["track{0}". format(index)]["duplicates"]["num_dup"] = dup

                    lib["track{0}". format(l)]["duplicates"]["dup_id"] = dup_id
                    lib["track{0}". format(index)]["duplicates"]["dup_id"] = dup_id

                    lib["track{0}". format(index)]["duplicates"]["dup_loc{0}".format(dup-1)] = index

                else:
                    lib["track{0}". format(l)]["duplicates"]["num_dup"] = dup

                lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(dup-1)] = index
                lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(dup)] = l

                dup = dup + 1
                # print(index, "+")
            elif i == same_tr_names-1 and dup == 0:
                Duplicate_Index.append(-1)
                # print(index, "-")

            last_ind = index+1

    else:
        Duplicate_Index.append(-1)

    dup = dup - 1
    iter = dup
    for x in range(dup):
        prev_tr = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x)]
        lib["track{0}". format(prev_tr)]["duplicates"]["num_dup"] = dup
        iter = iter - x

        for y in range (iter):
            a = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(x+y+1)]
            lib["track{0}". format(prev_tr)]["duplicates"]["dup_loc{0}".format(x+y+1)] = a
    # print(Duplicate_Index)

    dup = lib["track{0}". format(l)]["duplicates"]["num_dup"]

    if dup >= 1:
        for i in range(dup+1):
            loc = lib["track{0}". format(l)]["duplicates"]["dup_loc{0}".format(i)]
            track_uri = lib["track{0}". format(loc)]["uri"]
            lib["track{0}". format(l)]["uri{0}".format(i)] = track_uri
    else:
        lib["track{0}". format(l)]["uri0"] = track_uri

    l = l+1

print(lib["track0"]["duplicates"])
print(lib["track1"]["duplicates"])
print(lib["track2"]["duplicates"])
print(lib["track3"]["duplicates"])
print(lib["track4"]["duplicates"])
print(lib["track5"]["duplicates"])
print(lib["track6"]["duplicates"])
print(lib["track7"]["duplicates"])
print(lib["track8"]["duplicates"])
print(lib["track9"]["duplicates"])
print(lib["track10"]["duplicates"])

The Downfall of Us All
The Downfall Of Us All
Adam's Song
Adam's Song
Adam's Song
Alive
Alive
Alive
Alive
Alive
On The Floor
$outh $ide $uicide
Симпл димпл поп ит сквиш (Slowed + Reverb) [Remix]
น้ำแดงน้ำส้ม
{'dup_id': 0, 'num_dup': 1, 'dup_loc0': 0, 'dup_loc1': 1}
{'dup_id': 0, 'num_dup': 1, 'dup_loc0': 0, 'dup_loc1': 1}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 2, 'num_dup': 2, 'dup_loc0': 2, 'dup_loc1': 3, 'dup_loc2': 4}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': 6, 'num_dup': 1, 'dup_loc0': 6, 'dup_loc1': 8}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': 6, 'num_dup': 1, 'dup_loc0': 6, 'dup_loc1': 8}
{'dup_id': 5, 'num_dup': 2, 'dup_loc0': 5, 'dup_loc1': 7, 'dup_loc2': 9}
{'dup_id': -1, 'num_dup': 0}


In [533]:
# Number of [non-duplicate songs, duplicates of song 3, duplicates of song 5,...]
# num_uniques [#nd, #song3, #song5...]
num_uniques = []

# Labeling uniques and duplicates
# uniques = [-1, 1 , 2...]
uniques = list(set(Duplicate_Index))
uniques.sort()

for i in range (len(uniques)):
    count = Duplicate_Index.count(uniques[i])
    num_uniques.append(count)

print(Duplicate_Index)
print(uniques)
print(num_uniques)



[0, 0, 2, 2, 2, 5, 6, 5, 6, 5, -1, -1, -1, -1]
[-1, 0, 2, 5, 6]
[4, 2, 3, 3, 2]


In [534]:
# df = pd.DataFrame(lib)
df = pd.DataFrame.from_dict(lib, orient='index')
df2 = df['artists_info'].apply(pd.Series)
df3 = pd.concat([df.drop(['artists_info'], axis=1), df2], axis=1)

# Sort Alphabetically by Artist -> Album -> Song
df3 = df3.sort_values(['artists', 'album', 'song'], ascending = True, key = lambda col: col.str.lower())

# Track IDs
sl_tr_id = list(range(1,len(sl_tr_name)+1))
df3['id'] = sl_tr_id

df3.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist.csv', encoding='utf-8', index = False)

# Popen('C:\\Users\\mackt\\Python\\Music Library\\Data\\Playlist.csv', shell=True)
os.startfile('C:\\Users\\mackt\\Python\\Music Library\\Data\\Playlist.csv')

In [535]:
# # Sort Alphabetically by Artist -> Album -> Song
# df3 = df3.sort_values(['artists', 'album', 'song'], ascending = True, key = lambda col: col.str.lower())

# # Track IDs
# sl_tr_id = list(range(1,len(sl_tr_name)+1))
# df3['id'] = sl_tr_id

# # Creating CSV of Entire Playlist
# df3.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist.csv',index = False)

# df3 = df3.sort_index()

In [536]:
# Change the dates of all duplicate songs to the earliest release date
dup_date = {}

for i in range(len(Duplicate_Index)):
    dup_id = lib["track{0}". format(i)]["duplicates"]["dup_id"]
    tr_release = lib["track{0}". format(i)]["date_spot"]

    if dup_id != -1:
        if dup_id in dup_date:
            if tr_release > dup_date[dup_id]:
                tr_release = dup_date[dup_id]
            else:
                dup_date[dup_id] = tr_release
        else:
            dup_date[dup_id] = tr_release

for i in range(len(Duplicate_Index)):
    dup_id = lib["track{0}". format(i)]["duplicates"]["dup_id"]

    if dup_id != -1:
        lib["track{0}". format(i)]["date_spot"] = dup_date[dup_id]

# print(dup_date)

In [537]:
df = pd.DataFrame.from_dict(lib, orient='index')
df2 = df['duplicates'].apply(pd.Series)
df2 = df2[['dup_id', 'num_dup']]
df3 = df['artists_info'].apply(pd.Series)
df4 = pd.concat([df.drop(['artists_info'], axis=1), df2, df3], axis=1)

# Dropping Duplicates
df4 = df4.sort_values(['popularity'], ascending = False)

df4 = df4[(~df4.duplicated(['dup_id']) | (df4["dup_id"] == -1))]
# df4 = df4[(~(df4.duplicated(['dup_id']) & (df4["dup_id"] != -1)))]
df4 = df4.sort_values(['artists', 'album', 'song'], ascending = True, key = lambda col: col.str.lower())

# df.drop(['dup_id', 'song_low'], axis=1, inplace=True)

# Track IDs
sl_tr_id = list(range(1,len(df4)+1))
df4['id'] = sl_tr_id

# Creating CSV of Entire Playlist Without Duplicates
df4.to_csv(r'C:\Users\mackt\Python\Music Library\Data\Playlist_No_Duplicates.csv',index = False, encoding='utf-8')
os.startfile('C:\\Users\\mackt\\Python\\Music Library\\Data\\Playlist_No_Duplicates.csv')